Team 5 Project

Source Code: [link text](https://github.com/ckran/bikeshare/blob/main/availability.ipynb)

linear Regression: https://realpython.com/linear-regression-in-python/#python-packages-for-linear-regression

PCA: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html samples -> days,m varables, days

Source project: https://towardsdatascience.com/estimating-bike-availability-from-nyc-bike-share-data-7cfc4655d5f6

Dataset: https://s3.amazonaws.com/tripdata/202009-citibike-tripdata.csv.zip

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import timedelta
from matplotlib.dates import DateFormatter

In [ ]:
df = pd.read_csv('/content/202009-citibike-tripdata.csv',\
                 usecols=['starttime','start station id','stoptime','end station id'],\
                 parse_dates=['starttime','stoptime'])

FileNotFoundError: ignored

In [ ]:
df.info()

In [ ]:
dfrebal=pd.read_parquet('/content/202009-citibike-reblance.parquet')

In [ ]:
dfrebal.info()

In [ ]:
df = pd.concat([df,dfrebal])
df.reset_index(drop=True, inplace=True)
df

In [ ]:
dfs=df[['starttime','start station id']].assign(act=-1)

In [ ]:
dfe=df[['stoptime','end station id']].assign(act=1)

In [ ]:
dfs.columns=['docktime','stationid','act']
dfe.columns=['docktime','stationid','act']
dfse=pd.concat([dfs,dfe])

In [ ]:
dfse.sort_values(by=['docktime'], inplace=True)
dfse.reset_index(drop=True, inplace=True)
dfse.head(100)

In [ ]:
dfstations = \
  pd.read_csv('/content/202009-citibike-tripdata.csv',\
  usecols=['start station id','start station name']).\
  drop_duplicates()
dfstations.columns=['stationid','station name']
dfstations.set_index('stationid' ,  inplace=True)

In [ ]:
dfstations

In [ ]:
station = 'W 116 St & Broadway'
sid = dfstations.loc[dfstations['station name']==station].index[0]

In [ ]:
dfstation = dfse.loc[(dfse.stationid==sid) ]
dfstation.reset_index(drop=True, inplace=True)

In [ ]:
dfstation = dfstation.assign(cnt = dfstation.act.cumsum())
dfstation.head(10)

In [ ]:
dfstation.at[0, 'act'] =+ abs(dfstation.act.cumsum().min())

In [ ]:
dfstation = dfstation.assign(cnt = dfstation.act.cumsum())
dfstation.head(10)

In [ ]:
#adding day of week and hour columns

In [ ]:
sns.set(font_scale = 1)
plt.figure(figsize=(20,8))
ax=sns.lineplot(data=dfstation , x='docktime', y='cnt' )
ax.set_xlabel('Day')
ax.set_ylabel('Available Bikes') ;

In [ ]:
def availability (station,day):
    # inputs: station name, day
    # requires: dfstations, dfse
    sid = dfstations.loc[dfstations['station name']==station].index[0] # lookup station id
    dfstation = dfse.loc[(dfse.stationid==sid) ] # create dataframe for a specified station
    dfstation.reset_index(drop=True, inplace=True)
    dfstation = dfstation.assign(cnt = dfstation.act.cumsum()) # get running total
    dfstation.at[0, 'act'] =+ abs(dfstation.act.cumsum().min()) # find sub-zero bike count
    dfstation = dfstation.assign(cnt = dfstation.act.cumsum()) # recalculate running total

    # Create chart
    sns.set(font_scale = 2)
    plt.figure(figsize=(20,8))
    ax=sns.lineplot(data=dfstation.loc[dfstation.docktime.dt.day == day] , x='docktime', y='cnt' )
    ax.set_ylabel('Available Bikes')
    ax.set_xlabel('Hour of Day')
    ax.set_title('Citi Bike Station - ' + station +' - September ' + str(day))
    ax.xaxis.set_major_formatter(DateFormatter("%H")) ;

In [ ]:
#list(dfstations['station name'])

In [ ]:
availability ('W 52 St & 6 Ave',8)

In [ ]:
availability ('1 Ave & E 18 St', 9)

In [ ]:
availability ('W 116 St & Broadway', 9)

In [ ]:
print("New table with day and hour")


dfModified = dfstation.copy()
hour = []
weekday = []
monthday = []
for index, row in dfModified.iterrows():
    hour.append(row[dfModified.columns[0]].hour)
    weekday.append(row[dfModified.columns[0]].weekday())
    monthday.append(str(row[dfModified.columns[0]])[8:10])

#print(str(dfModified.iloc[1,0])[8:10]) #day is caharacters 9 adn 10

dfModified['hour'] = hour
dfModified['weekday'] = weekday
dfModified['monthday'] = weekday

#dfModified.head(10)
print(dfModified)

#num_rows = dfModified.shape[0]
#print(num_rows)
#create matrix with 30 rows and 25 columns, 24 for hours of the day adn one for day of the week

#get station id for 116 abnd broadway

New table with day and hour
                    docktime  stationid  act  cnt  hour  weekday  monthday
0    2020-09-01 01:47:31.845       3536    9    9     1        1         1
1    2020-09-01 02:00:29.753       3536   -1    8     2        1         1
2    2020-09-01 08:03:42.570       3536   -1    7     8        1         1
3    2020-09-01 08:11:30.764       3536    1    8     8        1         1
4    2020-09-01 08:31:01.706       3536   -1    7     8        1         1
...                      ...        ...  ...  ...   ...      ...       ...
3540 2020-09-30 22:41:03.506       3536    1    7    22        2         2
3541 2020-09-30 22:41:13.255       3536   -1    6    22        2         2
3542 2020-09-30 22:53:16.366       3536   -1    5    22        2         2
3543 2020-09-30 23:52:22.676       3536   -1    4    23        2         2
3544 2020-10-01 00:02:30.178       3536    1    5     0        3         3

[3545 rows x 7 columns]


In [ ]:
""" go through dfModified table
currenthour = 0 (will go 0-24)
currentDay = 1 (go 1-30)

while(currentDay <= 30)

  create empty hash map

  iterate through rows of dfModified,
    if day = current,
      if hour in hash map, add to list
      if hour not in hash map, create new list in map

  add count to sum, add 1 to denom

  for each key value of map,
    average = sum elemnts in list/size of list

  add to new dataset = date, currentDay, 0 , 1, ....
"""

currentHour = 0
currentDay = 1
currentWeekday = 0

while currentDay <= 30:

  map = {}

  for index, row in dfModified.iterrows():
    #creating map for day
    if row['monthday'] is currentDay:
      if row['hour'] in map:
        map[hour].append([row['cnt']])
      else:
        map[hour] = [row['cnt']]

  #creating averages list
  print("#creating averages list")
  averages = []
  for key in map.keys():
      value_list = map[key]
      value_sum = sum(value_list)
      averages.append(value_sum/len(value_list))
      print(value_sum + "plus" + (value_list) + " = " + value_sum/len(value_list))

    dfPCA = pd.DataFrame(columns=['date', 'day of week', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'])
    row_data = {'date': currentDay, 'day of week': , '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']}
    dfPCA = dfPCA.append(row_data, ignore_index=True)
    print(dfPCA)


    #if there is no data at that time then we need to maintain the previous sum

    #where you left off: adding


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df)